In [1]:
%matplotlib notebook

import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import h5py

from scipy.signal import tukey, butter, filtfilt
from scipy.stats import pearsonr

from time import time

from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.metrics import peak_signal_noise_ratio

from bm3d import bm3d

In [2]:
""" Setting random seeds """
from models import seed

# Python
import random as python_random
python_random.seed(seed)

# NumPy (random number generator used for sampling operations)
rng = np.random.default_rng(seed)

# Auxiliary functions

In [3]:
def butter_bandpass(lowcut, highcut, fs, order=2):
    
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    
    if low < 0:
        Wn = high
        btype = "lowpass"
    elif high < 0:
        Wn = low
        btype = "highpass"
    else:
        Wn = [low, high]
        btype = "bandpass"

    b, a = butter(order, Wn, btype=btype)
    
    return b, a


def taper_filter(arr, fmin, fmax, samp_DAS):
    b_DAS, a_DAS = butter_bandpass(fmin, fmax, samp_DAS)
    window_time = tukey(arr.shape[1], 0.1)
    arr_wind = arr * window_time
    arr_wind_filt = filtfilt(b_DAS, a_DAS, arr_wind, axis=-1)
    return arr_wind_filt


def xcorr(x, y):
    
    # FFT of x and conjugation
    X_bar = np.fft.rfft(x).conj()
    Y = np.fft.rfft(y)
    
    # Compute norm of data
    norm_x_sq = np.sum(x**2)
    norm_y_sq = np.sum(y**2)
    norm = np.sqrt(norm_x_sq * norm_y_sq)
    
    # Correlation coefficients
    R = np.fft.irfft(X_bar * Y) / norm
    
    # Return correlation coefficient
    return np.max(R)

def compute_xcorr_window(x):
    Nch = x.shape[0]
    Cxy = np.zeros((Nch, Nch)) * np.nan
    
    for i in range(Nch):
        for j in range(i):
            Cxy[i, j] = xcorr(x[i], x[j])
    
    return np.nanmean(Cxy)

def compute_moving_coherence(data, bin_size):
    
    N_ch = data.shape[0]
    
    cc = np.zeros(N_ch)
    
    for i in range(N_ch):
        start = max(0, i - bin_size // 2)
        stop = min(i + bin_size // 2, N_ch)
        ch_slice = slice(start, stop)
        cc[i] = compute_xcorr_window(data[ch_slice])
        
    return cc

# Load data

In [4]:
cwd = os.getcwd()
data_dir = os.path.join(cwd, "data")

HCMR_data_file = os.path.join(data_dir, "events_HCMR.h5")
HCMR_rec_file = os.path.join(data_dir, "reconstructions_HCMR_retrained.h5")

nestor_data_file = os.path.join(data_dir, "events_Nestor.h5")
nestor_rec_file = os.path.join(data_dir, "reconstructions_Nestor_retrained.h5")

HCMR_test_inds = [5, 10, 14, 19]
nestor_test_inds = [0, 4]

samp = 50.
gauge = 19.2

fmin = 1.0
fmax = 10.0


# Load/normalise HCMR data
with h5py.File(HCMR_data_file, "r") as h5f:
    HCMR_data = h5f["strain_rate"][...]
       
with h5py.File(HCMR_rec_file, "r") as h5f:
    HCMR_recs = h5f["strain_rate"][...]

HCMR_scales = np.zeros(HCMR_data.shape[:2])
for n in range(HCMR_data.shape[0]):
    for i in range(HCMR_data.shape[1]):
        HCMR_scales[n, i] = HCMR_data[n, i].std()
        HCMR_data[n, i] = HCMR_data[n, i] / HCMR_scales[n, i]

# Load/normalise NESTOR data
with h5py.File(nestor_data_file, "r") as h5f:
    nestor_data = h5f["strain_rate"][...]
    
with h5py.File(nestor_rec_file, "r") as h5f:
    nestor_recs = h5f["strain_rate"][...]

nestor_scales = np.zeros(nestor_data.shape[:2])
for n in range(nestor_data.shape[0]):
    for i in range(nestor_data.shape[1]):
        nestor_scales[n, i] = nestor_data[n, i].std()
        nestor_data[n, i] = nestor_data[n, i] / nestor_scales[n, i]

# Figure 6: Results HCMR data

## Select and process events

In [5]:
n_high = HCMR_test_inds[2]
n_low = HCMR_test_inds[0]

Nch, Nt = HCMR_data[n_high].shape

""" Bandpass filter reconstructions """
rec_filt_high = taper_filter(HCMR_recs[n_high], fmin=fmin, fmax=fmax, samp_DAS=samp)
rec_filt_low = taper_filter(HCMR_recs[n_low], fmin=fmin, fmax=fmax, samp_DAS=samp)

bin_size = 11

""" Compute coherence along fibre """

cc_HCMR_high = compute_moving_coherence(HCMR_data[n_high], bin_size)
cc_rec_high = compute_moving_coherence(rec_filt_high, bin_size)

cc_gain_high = cc_rec_high / cc_HCMR_high

cc_HCMR_low = compute_moving_coherence(HCMR_data[n_low], bin_size)
cc_rec_low = compute_moving_coherence(rec_filt_low, bin_size)

cc_gain_low = cc_rec_low / cc_HCMR_low

## Create figure

In [6]:
dist = np.arange(Nch) * gauge * 1e-3

imshow_params = {
    "vmin": -0.5,
    "vmax": 0.5,
    "aspect": "auto",
    "interpolation": "antialiased",
    "cmap": "viridis",
    "extent": [0, Nt/samp, Nch*gauge*1e-3, 0],
    "rasterized": True,
}

letters = "abcdefghijk"
letter_params = {
    "fontsize": 10,
    "verticalalignment": "top",
    "bbox": {"edgecolor": "k", "linewidth": 1, "facecolor": "w",}
}


# Draw canvas
fig = plt.figure(figsize=(9, 6), dpi=150)
gs = fig.add_gridspec(2, 7)

axes = []

""" Panel a """
ax = fig.add_subplot(gs[0, 0:2])
axes.append(ax)
ax.imshow(HCMR_data[n_high], **imshow_params)
ax.set_ylabel("Distance along cable [km]")
ax.set_title("Input data", pad=20)
ax.xaxis.set_ticklabels([])

""" Panel b """
ax = fig.add_subplot(gs[0, 2:4])
axes.append(ax)
ax.imshow(HCMR_recs[n_high], **imshow_params)
ax.set_title("Reconstruction", pad=20)
ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])

""" Panel c """
ax = fig.add_subplot(gs[0, 4:6])
axes.append(ax)
ax.imshow(rec_filt_high, **imshow_params)
ax.set_title("Reconstruction (filtered)", pad=20)
ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])

""" Panel d """
ax = fig.add_subplot(gs[0, -1])
axes.append(ax)
ax.plot(cc_gain_high, dist, c="k")
ax.axvline(1, ls=":", c="gray")
ax.set_xlim((0, 5))
ax.set_ylim((dist.max(), dist.min()))
ax.yaxis.set_ticklabels([])
ax.xaxis.set_ticklabels([])

""" Panel e """
ax = fig.add_subplot(gs[1, 0:2])
axes.append(ax)
ax.imshow(HCMR_data[n_low], **imshow_params)
ax.set_ylabel("Distance along cable [km]")
ax.set_xlabel("Time [s]")

""" Panel f """
ax = fig.add_subplot(gs[1, 2:4])
axes.append(ax)
ax.imshow(HCMR_recs[n_low], **imshow_params)
ax.set_xlabel("Time [s]")
ax.yaxis.set_ticklabels([])

""" Panel g """
ax = fig.add_subplot(gs[1, 4:6])
axes.append(ax)
ax.imshow(rec_filt_low, **imshow_params)
ax.set_xlabel("Time [s]")
ax.yaxis.set_ticklabels([])

""" Panel h """
ax = fig.add_subplot(gs[1, -1])
axes.append(ax)
ax.plot(cc_gain_low, dist, c="k")
ax.axvline(1, ls=":", c="gray")
ax.set_xlim((0, 5))
ax.set_ylim((dist.max(), dist.min()))
ax.yaxis.set_ticklabels([])
ax.set_xlabel("CC gain [-]")

# Add panel letters
for i, ax in enumerate(axes):
    ax.text(x=0.0, y=1.0, transform=ax.transAxes, s=letters[i], **letter_params)
    
# Adjust panel positions
plt.subplots_adjust(top=0.9, bottom=0.1, left=0.07, right=0.97, hspace=0.1, wspace=0.3)

plt.savefig("figures/HCMR_data.pdf")
plt.show()

<IPython.core.display.Javascript object>

# Figure 8: wiggle-for-wiggle comparison

In [7]:
# Selection of channels
selection = (25, 205, 562)

# Event selection
event = HCMR_test_inds[2]
rec_filt = taper_filter(HCMR_recs[event], fmin=fmin, fmax=fmax, samp_DAS=samp)

# Define time window
tmin = 10
tcor = 25
tmax = 38
t = np.arange(Nt) / samp

start = np.argmin(np.abs(t-tcor))
end = np.argmin(np.abs(t-tmax))
t_slice = slice(start, end)

# Draw canvas
fig, axes = plt.subplots(nrows=3, figsize=(5, 6))

# Loop over selected channels
for i, n in enumerate(selection):
    ax = axes[i]
    
    # Compute CC
    CC = pearsonr(HCMR_data[event, n, t_slice], rec_filt[n, t_slice])[0]
    
    # Plot wiggles
    ax.plot(t, HCMR_data[event, n], "k", lw=1.0, label="Original")
    ax.plot(t, rec_filt[n], "r", lw=1.0, label="Reconstruction (filtered)")
    ax.set_xlim((tmin, tmax))
    ax.set_yticks([])
    
    # Indicate channel and CC
    ax.set_title("Channel %d (%.1f km)  -  CC = %.3f" % (n, n * gauge * 1e-3, CC), fontsize=10)
    for spine in ("top", "left", "right"):
        ax.spines[spine].set_visible(False)

# Add x-axis label
ax.set_xlabel("Time [s]")

# Add legend
axes[0].legend(ncol=2, loc="upper center", bbox_to_anchor=(0.5, 1.7), frameon=False)

# Adjust subplots
plt.subplots_adjust(top=0.87, bottom=0.09, left=0.02, right=0.97, hspace=0.6)

plt.savefig("figures/wiggles_HCMR.pdf")
plt.show()

<IPython.core.display.Javascript object>

# Figure 9: compare with NLM/BM3D

## Perform NLM/BM3D denoising

In [8]:
# Event selection
event = HCMR_test_inds[2]
rec_filt = taper_filter(HCMR_recs[event], fmin=fmin, fmax=fmax, samp_DAS=samp)

# Estimate standard deviation of the noise
sigma_high = HCMR_data[event][:, :Nt//4].std()

# NLM denoising
t0 = time()
nlm_high = denoise_nl_means(HCMR_data[event], h=0.8*sigma_high, sigma=sigma_high, fast_mode=True, multichannel=False)

# BM3D denoising
t1 = time()
bm3d_high = bm3d(HCMR_data[event], sigma_high)
t2 = time()

print("Runtime \t NLM: %.2f \t BM3D: %.2f" % (t1-t0, t2-t1))

Runtime 	 NLM: 5.05 	 BM3D: 28.68


## Create figure

In [9]:
dist = np.arange(Nch) * gauge * 1e-3
time = np.arange(Nt) / samp

ch_start, ch_end = 0, 50
ch_slice = slice(ch_start, ch_end)

t_start, t_end = 750, 1800
t_slice = slice(t_start, t_end)

imshow_params = {
    "vmin": -0.5,
    "vmax": 0.5,
    "aspect": "auto",
    "interpolation": "antialiased",
    "cmap": "viridis",
    "extent": [time.min(), time.max(), dist.max(), dist.min()],
    "rasterized": True,
}

imshow_params2 = {
    "vmin": -0.5,
    "vmax": 0.5,
    "aspect": "auto",
    "interpolation": "antialiased",
    "cmap": "viridis",
    "extent": [time[t_slice].min(), time[t_slice].max(), dist[ch_slice].max(), dist[ch_slice].min()],
    "rasterized": True,
}

patch_params = {
    "xy": (time[t_start], dist[ch_start]), 
    "width": time[t_end]-time[t_start], 
    "height": dist[ch_end]-dist[ch_start],
    "fc": "none",
    "ec": "r",
    "lw": 1,
    "clip_on": False,
    "zorder": 999,
}

letters = "abcdefghijk"
letter_params = {
    "fontsize": 10,
    "verticalalignment": "top",
    "bbox": {"edgecolor": "k", "linewidth": 1, "facecolor": "w",}
}

# Draw canvas
fig = plt.figure(figsize=(9, 5), constrained_layout=True, dpi=150)
gs = fig.add_gridspec(2, 4)

axes = []

""" Panel a: input data """
ax = fig.add_subplot(gs[0, 0])
axes.append(ax)
ax.imshow(HCMR_data[n_high], **imshow_params)
ax.set_ylabel("Distance along cable [km]")
ax.set_title("Input data", pad=20)
ax.add_patch(Rectangle(**patch_params))

""" Panel b: NLM denoised """
ax = fig.add_subplot(gs[0, 1])
axes.append(ax)
ax.imshow(nlm_high, **imshow_params)
ax.set_title("NLM", pad=20)
ax.yaxis.set_ticklabels([])

""" Panel c: BM3D denoised """
ax = fig.add_subplot(gs[0, 2])
axes.append(ax)
ax.imshow(bm3d_high, **imshow_params)
ax.set_title("BM3D", pad=20)
ax.yaxis.set_ticklabels([])

""" Panel d: J-invariant denoised """
ax = fig.add_subplot(gs[0, 3])
axes.append(ax)
ax.imshow(rec_filt_high, **imshow_params)
ax.set_title("J-invariant (ours)", pad=20)
ax.yaxis.set_ticklabels([])

""" Panel e: zoom of panel a """
ax = fig.add_subplot(gs[1, 0])
axes.append(ax)
ax.imshow(HCMR_data[n_high][ch_slice, t_slice], **imshow_params2)
ax.set_ylabel("Distance along cable [km]")
ax.set_xlabel("Time [s]")

""" Panel f: zoom of panel b """
ax = fig.add_subplot(gs[1, 1])
axes.append(ax)
ax.imshow(nlm_high[ch_slice, t_slice], **imshow_params2)
ax.set_xlabel("Time [s]")
ax.yaxis.set_ticklabels([])

""" Panel g: zoom of panel c """
ax = fig.add_subplot(gs[1, 2])
axes.append(ax)
ax.imshow(bm3d_high[ch_slice, t_slice], **imshow_params2)
ax.set_xlabel("Time [s]")
ax.yaxis.set_ticklabels([])

""" Panel h: zoom of panel d """
ax = fig.add_subplot(gs[1, 3])
axes.append(ax)
ax.imshow(rec_filt_high[ch_slice, t_slice], **imshow_params2)
ax.set_xlabel("Time [s]")
ax.yaxis.set_ticklabels([])

# Add panel letters
for i, ax in enumerate(axes):
    ax.text(x=0.0, y=1.0, transform=ax.transAxes, s=letters[i], **letter_params)

plt.savefig("figures/HCMR_NLM_BM3D.pdf")
plt.show()

<IPython.core.display.Javascript object>

# Figure 7: Results NESTOR data

## Select and process events

In [10]:
n_high = nestor_test_inds[0]
n_low = nestor_test_inds[1]

Nch, Nt = nestor_data[n_high].shape

""" Bandpass filter reconstructions """
rec_filt_high = taper_filter(nestor_recs[n_high], fmin=fmin, fmax=fmax, samp_DAS=samp)
rec_filt_low = taper_filter(nestor_recs[n_low], fmin=fmin, fmax=fmax, samp_DAS=samp)

bin_size = 11

""" Compute coherence along fibre """

cc_nestor_high = compute_moving_coherence(nestor_data[n_high], bin_size)
cc_rec_high = compute_moving_coherence(rec_filt_high, bin_size)

cc_gain_high = cc_rec_high / cc_nestor_high

cc_nestor_low = compute_moving_coherence(nestor_data[n_low], bin_size)
cc_rec_low = compute_moving_coherence(rec_filt_low, bin_size)

cc_gain_low = cc_rec_low / cc_nestor_low

## Create figure

In [11]:
dist = np.arange(Nch) * gauge * 1e-3

imshow_params = {
    "vmin": -0.5,
    "vmax": 0.5,
    "aspect": "auto",
    "interpolation": "antialiased",
    "cmap": "viridis",
    "extent": [0, Nt/samp, Nch*gauge*1e-3, 0],
    "rasterized": True,
}

letters = "abcdefghijk"
letter_params = {
    "fontsize": 10,
    "verticalalignment": "top",
    "bbox": {"edgecolor": "k", "linewidth": 1, "facecolor": "w",}
}


# Draw canvas
fig = plt.figure(figsize=(9, 6), dpi=150)
gs = fig.add_gridspec(2, 7)


axes = []

""" Panel a """
ax = fig.add_subplot(gs[0, 0:2])
axes.append(ax)
ax.imshow(nestor_data[n_high], **imshow_params)
ax.set_ylabel("Distance along cable [km]")
ax.set_title("Input data", pad=20)
ax.xaxis.set_ticklabels([])

""" Panel b """
ax = fig.add_subplot(gs[0, 2:4])
axes.append(ax)
ax.imshow(nestor_recs[n_high], **imshow_params)
ax.set_title("Reconstruction", pad=20)
ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])

""" Panel c """
ax = fig.add_subplot(gs[0, 4:6])
axes.append(ax)
ax.imshow(rec_filt_high, **imshow_params)
ax.set_title("Reconstruction (filtered)", pad=20)
ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])

""" Panel d """
ax = fig.add_subplot(gs[0, -1])
axes.append(ax)
ax.plot(cc_gain_high, dist, c="k")
ax.axvline(1, ls=":", c="gray")
ax.set_xlim((0, 5))
ax.set_ylim((dist.max(), dist.min()))
ax.yaxis.set_ticklabels([])
ax.xaxis.set_ticklabels([])

""" Panel e """
ax = fig.add_subplot(gs[1, 0:2])
axes.append(ax)
ax.imshow(nestor_data[n_low], **imshow_params)
ax.set_ylabel("Distance along cable [km]")
ax.set_xlabel("Time [s]")

""" Panel f """
ax = fig.add_subplot(gs[1, 2:4])
axes.append(ax)
ax.imshow(nestor_recs[n_low], **imshow_params)
ax.set_xlabel("Time [s]")
ax.yaxis.set_ticklabels([])

""" Panel g """
ax = fig.add_subplot(gs[1, 4:6])
axes.append(ax)
ax.imshow(rec_filt_low, **imshow_params)
ax.set_xlabel("Time [s]")
ax.yaxis.set_ticklabels([])

""" Panel h """
ax = fig.add_subplot(gs[1, -1])
axes.append(ax)
ax.plot(cc_gain_low, dist, c="k")
ax.axvline(1, ls=":", c="gray")
ax.set_xlim((0, 5))
ax.set_ylim((dist.max(), dist.min()))
ax.yaxis.set_ticklabels([])
ax.set_xlabel("CC gain [-]")

# Add panel letters
for i, ax in enumerate(axes):
    ax.text(x=0.0, y=1.0, transform=ax.transAxes, s=letters[i], **letter_params)
    
# Adjust panel positions
plt.subplots_adjust(top=0.9, bottom=0.1, left=0.07, right=0.97, hspace=0.1, wspace=0.3)

plt.savefig("figures/nestor_data.pdf")
plt.show()

<IPython.core.display.Javascript object>